In [ ]:
from oai_tool import tool
from typing import Annotated
from pydantic import Field
from enum import Enum
import json
import openai
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
client = openai.OpenAI(api_key=OPENAI_API_KEY)
GPT_MODEL = "gpt-4o"

def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [ ]:
class TemperatureUnit(Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"
        

@tool
def get_current_weather(
    location: Annotated[str, Field(description="The city and state, e.g. San Francisco, CA")],
    format: Annotated[TemperatureUnit, Field(description="The temperature unit to use. Infer this from the user's location.")]
) -> dict:
    """Get the current weather"""
    return json.dumps({
        "location": location,
        "temperature": "25°C" if format == TemperatureUnit.CELSIUS else "77°F",
        "condition": "Sunny"
    })

In [ ]:
print(json.dumps(get_current_weather._schema, indent=4))

In [ ]:
tools = [get_current_weather]
tool_dispatch = {tool._schema['function']['name']: tool for tool in tools}
messages = [
        {"role": "system", "content": "You fetch the current weather for a user."},
        {"role": "user", "content": "What's the weather in New York?"}
    ]
response = chat_completion_request(
    messages=messages,
    tools=[tool._schema for tool in tools],
    tool_choice="auto",
    model=GPT_MODEL
)

In [ ]:
tool_message = response.choices[0].message
messages.append(tool_message)
tool_call = response.choices[0].message.tool_calls[0]
tool_call

In [ ]:
tool_calls = response.choices[0].message.tool_calls
if tool_calls:
    # There could be multiple tool calls, but for this example we only expect one
    tool_call = tool_calls[0]
    tool_call_id = tool_call.id
    tool_name = tool_call.function.name
    tool = tool_dispatch[tool_name]
    # This is where the future validation logic code would go
    # For now, we just assume the tool call is valid
    tool_args = json.loads(tool_call.function.arguments)
    tool_response = tool(**tool_args)
    tool_message = {
        "role": "tool",
        "content": tool_response,
        "tool_call_id": tool_call_id,
        "name": tool_name,
    }
    
 

# raw_json_response = response.choices[0].message.tool_calls[0].function.arguments

In [ ]:

tool_message, tool_response

In [ ]:
messages_to_send = messages + [tool_message]
messages_to_send

In [ ]:
model_response_with_function_call = client.chat.completions.create(
    model="gpt-4o",
    messages=messages + [tool_message],
)

In [ ]:
model_response_with_function_call.choices[0].message.content